In [ ]:
data_1 = ["Flow_Lyngsåna", "Flow_Tapping_Kalltveit", "Flow_Tapping", "Flow_Lyngsvatn_Overflow",
    "Precipitation_Nilsebu", "Air_Temperature_Nilsebu",
    "Precipitation_Fister", "Air Temperature Fister",]
target = "Flow_Kalltveit"